# Generating Simpson's Paradox

We have been maually setting, but now we should also be able to generate it more programatically.  his notebook will describe how we develop some functions that will be included in the `sp_data_util` package.

In [3]:
# %load code/env
# standard imports we use throughout the project
import numpy as np
import pandas as pd
import seaborn as sns

import detect_simpsons_paradox as dsp
from sp_data_util import sp_plot

We have been thinking of SP hrough gaussian mixture data, so we'll first work wih that. To cause SP we need he clusters to have an opposite trend of the per cluster covariance. 